In [0]:
import pyspark.sql.functions as F
from pyspark.sql.functions import when, col
from pyspark.sql.types import NumericType, StringType
import pyspark.pandas as ps
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## importando dataframe spark

In [0]:
df = spark.read.csv(
    "/Volumes/telecom/bronze/teleco_raw/WA_Fn-UseC_-Telco-Customer-Churn.xls",
    header=True,
    inferSchema=True,
)

In [0]:
display(df.head(10))

Listando as colunas

In [0]:
df.columns

Aqui podemos observar que a coluna `TotalCharges` está sendo tratada como um `string` e não como um tipo númerico `float`

In [0]:
df.printSchema()


In [0]:
df = df.withColumn(
    "SeniorCitizen",
    when(col("SeniorCitizen") == 1, "Yes").otherwise("No")
)


In [0]:
display(df.describe())


É confirmado pois não  conseguimos extrair estatisticas numéricas desta coluna.

In [0]:
# Loop pelas colunas do DataFrame
for col_name, col_type in df.dtypes:
    if isinstance(df.schema[col_name].dataType, NumericType): # verifica se é numérica.
        print(f" Estatísticas da coluna '{col_name}':")
        df.select(col_name).describe().show()



Vamos retirar os espaço em branco de colunas do tipo string

In [0]:
for coluna in df.columns: # percorre todas as colunas
    if isinstance(df.schema[coluna].dataType, StringType): # verifica se a coluna é string
        df = df.withColumn(
            coluna,
            F.when(F.trim(F.col(coluna)) == "", None).otherwise(F.col(coluna)) #se há espaços em branco, substituir por NULL
        )

Após o tratamento, verificamos a ocorrência de 11 valores NULL em `TotalCharges` 

In [0]:
listagem_nulos = [(coluna, df.where(F.col(coluna).isNull()).count() )for coluna in df.columns]
display(listagem_nulos)

In [0]:

colunas = [x[0] for x in listagem_nulos]
valores = [x[1] for x in listagem_nulos]

plt.figure(figsize=(8,5))
plt.bar(colunas, valores, color='skyblue')
plt.title('Quantidade de valores nulos por coluna')
plt.xlabel('Colunas')

plt.xticks(rotation=-90)
plt.yticks(np.arange(0, max(valores)+1, 1))
plt.ylabel('Quantidade de nulos')
plt.tight_layout()
plt.show()

Agora precisamos:
- (1) transfomar `TotalCharges` para um tipo numérico e;
- (2) Eliminar as 11 linhas nulas - haja vista as poucas ocorrências

In [0]:
# transformando a coluna TotalCharges para double
df = df.withColumn("TotalCharges", F.col("TotalCharges").cast("double"))

In [0]:
df.printSchema()

In [0]:
# linhas e colunas
print((df.count(), len(df.columns)))

In [0]:
# retirando as linhas com valroes nulos
df = df.dropna(how = 'all', subset=['TotalCharges'])

In [0]:
print((df.count(), len(df.columns)))

In [0]:
# Ultima verificação
listagem_nulos = [(coluna, df.where(F.col(coluna).isNull()).count() )for coluna in df.columns]
display(listagem_nulos)

## Salvando o trabalho em formato de tabela

In [0]:
# formato delta para preservação de logs
df.write.format("delta") \
  .mode("overwrite") \
  .option("overwriteSchema", "true") \
  .save("/Volumes/telecom/silver/teleco_cleaned")



In [0]:
# Salvando em formato de tabela
df.write.format("delta") \
    .mode("overwrite") \
    .saveAsTable("telecom.silver.teleco_cleaned")


## Teste

In [0]:
%sql
SELECT * FROM telecom.silver.teleco_cleaned
LIMIT 4;